In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from gensim.models import Word2Vec, KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.neighbors import KNeighborsClassifier 

tf.random.set_seed(10)

In [2]:
train_x_np = np.load('./training_embeddings.npy', allow_pickle=True)
train_y_np = np.load('./training_labels.npy', allow_pickle=True)

test_x_np = np.load('./testing_embeddings.npy', allow_pickle=True)
test_y_np = np.load('./testing_labels.npy', allow_pickle=True)

In [3]:
test_x_np.shape

(2242, 49152)

In [4]:
len(test_y_np)

2242

In [5]:
train_y_np.shape

(5636,)

In [6]:
# train_df = pd.DataFrame({'Text':train_x_np, 'Category':train_y_np})

In [7]:
train_df = pd.read_csv('../../Task_2/preprocessed_training_data.csv')
test_df = pd.read_csv('../../Task_2/preprocessed_testing_data.csv')

In [8]:
# labels = {}

# for index, i in enumerate(test_df['Category'].unique()):
#     labels[i] = index

# labels

In [9]:
labels = {}

for index, i in enumerate(set(train_y_np)):
    labels[i] = index

labels

{'earn': 0, 'money-fx': 1, 'trade': 2, 'interest': 3, 'acq': 4, 'crude': 5}

In [10]:
def label_encode(label):
    return labels[label]

In [11]:
train_y_np

array(['earn', 'acq', 'earn', ..., 'money-fx', 'money-fx', 'money-fx'],
      dtype=object)

In [12]:
train_y_np = [label_encode(i) for i in train_y_np]
test_y_np = [label_encode(i) for i in test_y_np]

In [13]:
train_y_np

[0,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 4,
 0,
 0,
 4,
 0,
 5,
 4,
 0,
 0,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 0,
 5,
 5,
 5,
 0,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 3,
 3,
 3,
 5,
 5,
 5,
 5,
 1,
 5,
 1,
 4,
 2,
 5,
 0,
 3,
 4,
 0,
 4,
 4,
 3,
 4,
 0,
 3,
 4,
 4,
 3,
 2,
 3,
 2,
 0,
 5,
 5,
 5,
 2,
 0,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 4,
 0,
 4,
 4,
 0,
 4,
 0,
 0,
 4,
 0,
 4,
 0,
 4,
 4,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 3,
 4,
 0,
 0,
 0,
 4,
 0,
 4,
 0,
 0,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 4,
 0,
 4,
 4,
 3,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 4,
 4,
 4,
 5,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 3,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 4,
 0,
 4,
 0,
 4,
 4,
 0,
 4,
 0,
 0,
 0,
 4,
 4,
 0,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 5,
 4,
 0,
 4,
 4,
 4,


In [14]:
# train_df['Category'] = train_df['Category'].apply(label_encode)

In [15]:
# test_df['Category'] = test_df['Category'].apply(label_encode)

In [16]:
# train_df

In [17]:
# x_train = train_df['Text']
# y_train = train_df['Category']

In [18]:
x_train = train_x_np
y_train = train_y_np

x_test = test_x_np
y_test = test_y_np

In [19]:
# x_test = test_df['Text']    

In [20]:
# y_test = test_df['Category']

In [21]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

In [22]:
from tensorflow.keras.utils import to_categorical
enc_y_train = to_categorical(y_train)
enc_y_test = to_categorical(y_test)
enc_y_val = to_categorical(y_val)


In [23]:
enc_y_train[0]

array([0., 0., 0., 0., 1., 0.], dtype=float32)

In [24]:
# Build the ANN model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(49152,)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# model.add(Dense(16, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

# model.add(Dense(16, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))

model.add(Dense(len(labels), activation='softmax'))  # 10 class for Reuters dataset

callback = EarlyStopping(monitor='accuracy', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)


# Compile the model
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with validation data
history = model.fit(x_train, enc_y_train,
                    epochs=100,
                    batch_size=32,
                    validation_data=(x_val, enc_y_val), callbacks=[callback, reduce_lr])





Epoch 1/100


124/124 [==============================] - 3s 17ms/step - loss: 1.3651 - accuracy: 0.5450 - val_loss: 3.2748 - val_accuracy: 0.5038 - lr: 0.0010
Epoch 2/100
124/124 [==============================] - 2s 15ms/step - loss: 1.0669 - accuracy: 0.6527 - val_loss: 13.9448 - val_accuracy: 0.0591 - lr: 0.0010
Epoch 3/100
124/124 [==============================] - 2s 14ms/step - loss: 0.9862 - accuracy: 0.6705 - val_loss: 2.5142 - val_accuracy: 0.5038 - lr: 0.0010
Epoch 4/100
124/124 [==============================] - 2s 14ms/step - loss: 0.9470 - accuracy: 0.6728 - val_loss: 39.7689 - val_accuracy: 0.2809 - lr: 0.0010
Epoch 5/100
124/124 [==============================] - 2s 14ms/step - loss: 0.8843 - accuracy: 0.6910 - val_loss: 16.3741 - val_accuracy: 0.5038 - lr: 0.0010
Epoch 6/100
124/124 [==============================] - 2s 14ms/step - loss: 0.8617 - accuracy: 0.6928 - val_loss: 49.6719 - val_accuracy: 0.2809 - lr: 0.0010
Epoch 7/100
124/124 [=============================

In [25]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, enc_y_test)
print(f'Test accuracy: {test_acc:.4f}')

71/71 [==============================] - 0s 2ms/step - loss: 10.9395 - accuracy: 0.3118
Test accuracy: 0.3118


In [26]:
model = xgb.XGBClassifier(random_state = 42)

In [27]:
model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=10, verbose = True)

C:\Users\nawee\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [ ]:
predictions = model.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test, predictions)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, predictions))

Accuracy: 0.8050847457627118

Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.39      0.48        88
           1       0.69      0.33      0.44       144
           2       0.50      0.27      0.35       112
           3       0.88      0.96      0.92      1087
           4       0.71      0.49      0.58       112
           5       0.75      0.85      0.80       699

    accuracy                           0.81      2242
   macro avg       0.69      0.55      0.59      2242
weighted avg       0.79      0.81      0.79      2242



In [ ]:
lgr_clf = LogisticRegression(multi_class='multinomial', verbose=2, random_state=0)

In [ ]:
lgr_clf.fit(x_train, y_train)

C:\Users\nawee\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial', random_state=0, verbose=2)

In [ ]:
lgr_predictions = lgr_clf.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test, lgr_predictions)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, lgr_predictions))

Accuracy: 0.7261373773416593

Classification Report:
              precision    recall  f1-score   support

           0       0.42      0.12      0.19        88
           1       0.00      0.00      0.00       144
           2       0.40      0.04      0.07       112
           3       0.80      0.94      0.87      1087
           4       0.62      0.14      0.23       112
           5       0.63      0.83      0.72       699

    accuracy                           0.73      2242
   macro avg       0.48      0.34      0.35      2242
weighted avg       0.65      0.73      0.67      2242



C:\Users\nawee\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\nawee\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\nawee\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7) 

In [ ]:
knn.fit(x_train, y_train) 

KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn_predictions = knn.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test, knn_predictions)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, knn_predictions))

Accuracy: 0.7172167707404103

Classification Report:
              precision    recall  f1-score   support

           0       0.29      0.58      0.39        88
           1       0.53      0.38      0.44       144
           2       0.32      0.29      0.30       112
           3       0.80      0.96      0.87      1087
           4       0.51      0.29      0.37       112
           5       0.78      0.57      0.66       699

    accuracy                           0.72      2242
   macro avg       0.54      0.51      0.51      2242
weighted avg       0.72      0.72      0.71      2242

